<a href="https://colab.research.google.com/github/vamshap/PySpark-Challenges/blob/main/CountScoreofFreinds.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, count, sum

# Create Spark session
spark = SparkSession.builder.appName("FriendsScore").getOrCreate()

# Sample data for Friend table
friend_data = [
    (1, 2),
    (1, 3),
    (1, 4),
    (2, 5),
    (3, 1),
    (4, 2),
    (5, 3)
]
friend_columns = ["PersonID", "FriendID"]

# Sample data for Person table
person_data = [
    (1, 'Alice', 88),
    (2, 'Bob', 11),
    (3, 'Davis', 27),
    (4, 'Tara', 45),
    (5, 'John', 63)
]
person_columns = ["PersonID", "Name", "Score"]

# Create DataFrames
friend_df = spark.createDataFrame(friend_data, friend_columns)
person_df = spark.createDataFrame(person_data, person_columns)
friend_df.show()
person_df.show()


# Step 1: PersonAndFriends - Count number of friends
person_and_friends_df = (
    friend_df.join(person_df, friend_df.PersonID == person_df.PersonID)
    .groupBy(friend_df.PersonID, person_df.Name)
    .agg(count("*").alias("NoofFriends"))
)




# Step 2: FriendScore - Sum of scores of friends
friend_score_df = (
    friend_df.join(person_df, friend_df.FriendID == person_df.PersonID, "left")
    .groupBy(friend_df.PersonID)
    .agg(sum(person_df.Score).alias("FriendScore"))
)

# Step 3: Join and Filter
result_df = (
    person_and_friends_df.join(friend_score_df, "PersonID", "left")
    .filter(col("FriendScore") > 100)
    .select("PersonID", "FriendScore")
)

# Display result


+--------+--------+
|PersonID|FriendID|
+--------+--------+
|       1|       2|
|       1|       3|
|       1|       4|
|       2|       5|
|       3|       1|
|       4|       2|
|       5|       3|
+--------+--------+

+--------+-----+-----+
|PersonID| Name|Score|
+--------+-----+-----+
|       1|Alice|   88|
|       2|  Bob|   11|
|       3|Davis|   27|
|       4| Tara|   45|
|       5| John|   63|
+--------+-----+-----+

